In [10]:
import numpy as np
import xarray as xr
import pandas as pd
import time

### ===== EXTRAÇÃO DE DADOS MENSAIS ===== #

# set correct path of the netcdf files
path_var = 'C:\\Users\lucas.ramos\OneDrive\Desktop\Base Climatica\Tipos/'

# Posicoes: Colocar em ordem, separando por virgula. Neste exemplo temos dois pontos em que as coordenadas
# (lat, lon) sao (-20.6,-44.6) e  (-21.0, -44.1), respectivamente para o primeiro e segundo ponto.
# Pode-se colocar quantos pontos quiser, apenas separe por virgula.

#Região de Fazendas no Maranhão
lat = [-15.78944444]
lon = [-47.92583332]

# variables names
# var_names = ['Rs', 'u2','Tmax', 'Tmin', 'RH', 'pr', 'ETo']
var_names = ['pr', 'ETo']

# function to read the netcdf files
def rawData(var2get_xr, var_name2get):
    return var2get_xr[var_name2get].sel(longitude=xr.DataArray(lon, dims='z'),
                                          latitude=xr.DataArray(lat, dims='z'),
                                          method='nearest').values

# getting data from NetCDF files
for n, var_name2get in enumerate(var_names):
    print("getting " + var_name2get)
    if var_name2get in ["pr", "ETo"]:
        var2get_xr = xr.open_mfdataset(path_var + var_name2get + '*.nc').resample(time="M").sum("time")
        # var2get_xr[var_name2get].sel(latitude=lat[0], longitude=lon[0], method='nearest').plot()
    else:
        #var2get_xr = xr.open_mfdataset(path_var + var_name2get + '*.nc').resample(time="M").mean("time")
        print('aaaaaaaaa')

    if n == 0:
        var_ar = rawData(var2get_xr, var_name2get)
        n_lines = var_ar.shape[0]
        time = var2get_xr.time.values
    else:
        var_ar = np.c_[var_ar, rawData(var2get_xr, var_name2get)]

print(var2get_xr)
# saving
for n in range(len(lat)):
    name_file =  'lat{:.2f}_lon{:.2f}.csv'.format(lat[n], lon[n])
    print(f'arquivo {n + 1} de um total de {len(lat)}; nome do arquivo: {name_file}')
    if ~np.isnan(var_ar[0, n]):
        file = var_ar[:, n::len(lon)]
        pd.DataFrame(file, index=time, columns=var_names).to_csv(name_file, float_format='%.1f')

getting pr
getting ETo
<xarray.Dataset>
Dimensions:    (latitude: 393, longitude: 391, time: 235)
Coordinates:
  * latitude   (latitude) float32 -33.85 -33.75 -33.65 -33.55 ... 5.15 5.25 5.35
  * longitude  (longitude) float32 -73.85 -73.75 -73.65 ... -35.05 -34.95 -34.85
  * time       (time) datetime64[ns] 2001-01-31 2001-02-28 ... 2020-07-31
Data variables:
    ETo        (time, latitude, longitude) float32 dask.array<chunksize=(1, 393, 391), meta=np.ndarray>
Attributes:
    title:                Brazilian Daily Weather Gridded Data (1961-2020)
    institution:          Federal University of Espirito Santo; University of...
    institution_id:       UFES; UTEXAS
    history:              Created Tue May 31 16:03:02 2022
    temporal_resolution:  24-hourly
    contact:              alexandre.xavier@ufes.br; careyking@energy.utexas.e...
    main_code:            ETo.py
    update_information:   https://sites.google.com/site/alexandrecandidoxavie...
    main_paper:           Xavier, A.